In [1]:
import tensorflow as tf
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from tensorflow.keras.preprocessing.image import ImageDataGenerator

In [2]:
training_data_dir = '/tmp/cats-and-dogs/training_set/training_set/'
test_data_dir = '/tmp/cats-and-dogs/test_set/test_set/'

img_shape = (128, 128, 3)

datagen = ImageDataGenerator(rescale=1./255)
training_set = datagen.flow_from_directory(
    directory=training_data_dir, 
    target_size=(128, 128), 
    classes=['dogs', 'cats'], 
    class_mode='binary',
    batch_size=128
)
test_set = datagen.flow_from_directory(
    directory=test_data_dir, 
    target_size=(128, 128), 
    classes=['dogs', 'cats'], 
    class_mode='binary',
    batch_size=128
)


Found 8005 images belonging to 2 classes.
Found 2023 images belonging to 2 classes.


In [3]:
base_model = tf.keras.applications.MobileNetV2(input_shape=img_shape, include_top=False, weights='imagenet')
base_model.trainable=False
global_average_layer = tf.keras.layers.GlobalAveragePooling2D()(base_model.output)
prediction_layer = tf.keras.layers.Dense(units=1, activation='sigmoid')(global_average_layer)

In [4]:
model = tf.keras.models.Model(inputs=base_model.input, outputs=prediction_layer)
model.compile(optimizer=tf.keras.optimizers.RMSprop(learning_rate=0.0001), loss='binary_crossentropy', metrics=['accuracy'])

In [ ]:
model.fit_generator(generator=training_set, epochs=5, validation_data=test_set)

In [8]:
base_model.trainable = True
fine_tune_at = 100
for layer in base_model.layers[:fine_tune_at]:
    layer.trainable = False
    
model.compile(optimizer=tf.keras.optimizers.RMSprop(learning_rate=0.0001), loss='binary_crossentropy', metrics=['accuracy'])
model.fit_generator(generator=training_set, epochs=5, validation_data=test_set)

Epoch 1/5
63/63 [==============================] - 24s 295ms/step - loss: 0.1188 - accuracy: 0.9522 - val_loss: 0.1062 - val_accuracy: 0.9634
Epoch 2/5
63/63 [==============================] - 18s 282ms/step - loss: 0.0227 - accuracy: 0.9944 - val_loss: 0.0867 - val_accuracy: 0.9713
Epoch 3/5
63/63 [==============================] - 18s 284ms/step - loss: 0.0106 - accuracy: 0.9971 - val_loss: 0.1476 - val_accuracy: 0.9659
Epoch 4/5
63/63 [==============================] - 18s 282ms/step - loss: 0.0034 - accuracy: 0.9993 - val_loss: 0.1546 - val_accuracy: 0.9649
Epoch 5/5
63/63 [==============================] - 18s 283ms/step - loss: 0.0018 - accuracy: 0.9994 - val_loss: 0.1603 - val_accuracy: 0.9654
